## GIS 5572 Lab 2
This notebook is part of Lab 2, and reflects the decision about which temperature data is relevant to the final project. It includes the acquisition of the data. The intention of this is to use it in the app we are designing. 

I am working on how to get the results in to the Postgres database. (See notebook about the Census data.)

In [19]:
import csv
import requests
import json
import time
import pandas as pd
import time as t #datetime and time do not work together by default, change name of time to t
from datetime import datetime, timedelta
import urllib.parse


I am pulling the current weather data to go with the transportation data.
I found the documentation at https://www.weather.gov/documentation/services-web-api#/default/zone_list

In [2]:
alerts_url = 'https://api.weather.gov/alerts/active/zone/MNZ060'
response = requests.get(alerts_url) 

# Check if data request is successful
data = response.json()

# Extract and print alerts
if "features" in data:
    alerts = data["features"]
    if alerts:
        print("Current Weather for Hennepin County:")
        for alert in alerts:
            title = alert["properties"]["headline"]
            description = alert["properties"]["description"]
            print(f"{title}: {description}")
    else:
        print("No alerts")
else:
    print("No features in API response")

No alerts


In [3]:
# Pull the forecast
# MNZ060 is the zone code for Hennepin County
forecast_url = 'https://api.weather.gov/zones/forecast/MNZ060/forecast'
forecast_response = requests.get(forecast_url)
forecast_data = forecast_response.json()

# Extract and print forecast
if 'properties' in forecast_data and 'periods' in forecast_data['properties']:
    periods = forecast_data["properties"]["periods"]
    forecast = forecast_data['properties']['periods']
    if periods:
        print("Forecast for Hennepin County:")
        for period in periods:
            
            day_name = period['name']
            forecast_text = period['detailedForecast']
            print(f"{day_name} : {forecast_text}")
    else:
        print("No forecast")

# TO DO: Get this to just pull today's forecast and the next 2 days

Forecast for Hennepin County:
Tonight : Mostly cloudy. Lows around 40. Northeast winds 10 to 15 mph.
Thursday : Cooler. Mostly cloudy. Highs in the lower 50s. Northeast winds 10 to 15 mph.
Thursday Night : Mostly cloudy. Lows in the lower 30s. North winds 5 to 15 mph.
Friday : Mostly sunny. Highs in the lower 50s. Northwest winds 5 to 15 mph.
Friday Night : Breezy. Partly cloudy in the evening then becoming mostly cloudy. Lows in the upper 30s. Southwest winds 10 to 15 mph shifting to the west 15 to 25 mph after midnight.
Saturday : Windy. Partly sunny. Highs in the upper 40s.
Saturday Night : Colder. Partly cloudy. Lows in the mid 20s.
Sunday : Colder. Partly sunny. Highs in the mid 30s.
Sunday Night : Partly cloudy. Lows around 20.
Monday : Mostly sunny. Highs in the upper 30s.
Monday Night : Partly cloudy. Lows in the mid 20s.
Tuesday : Mostly sunny. Highs in the upper 40s.
Tuesday Night : Mostly clear. Lows in the upper 20s.
Wednesday : Sunny. Highs in the upper 40s.


In [15]:
#Pull in current date and time
# Pull in current date and time
now = datetime.now()
print("Current date and time:", now)

ftime = now.strftime('%m/%d/%y %H:%M:%S')
print(ftime) 




Current date and time: 2024-03-13 22:19:54.942925
03/13/24 22:19:54


## Current Weather Observations

In [20]:

# This section gets the current time, calcuates one hour later, and formats the times for the API call
# Get the current time 
current_time = datetime.utcnow()

# Calculate one hour later
one_hour_later = current_time + timedelta(hours=1)

# Format the times for the API call
start_time = current_time.strftime('%Y-%m-%dT%H:%M:%S.%fZ')
end_time = one_hour_later.strftime('%Y-%m-%dT%H:%M:%S.%fZ')

# URL encode the times
start_time_encoded = urllib.parse.quote(start_time)
end_time_encoded = urllib.parse.quote(end_time)

In [23]:
# Construct the API call
observation_url = f"https://api.weather.gov/zones/forecast/MNZ060/observations?start={start_time_encoded}&end={end_time_encoded}"

print("API URL:", observation_url)

API URL: https://api.weather.gov/zones/forecast/MNZ060/observations?start=2024-03-14T03%3A30%3A16.902151Z&end=2024-03-14T04%3A30%3A16.902151Z


In [24]:
# Make the API call using the URL we constructed
observation_response = requests.get(observation_url)
observation_data = observation_response.json()

# Extract and print observations
if "features" in observation_data:
    print("Current Weather Conditions for Hennepin County:")
    for feature in observation_data["features"]:
        properties = feature.get("properties", {})
        temperature = properties.get("temperature", {}).get("value")
        wind_speed = properties.get("windSpeed", {}).get("value")
    
        if temperature is not None and wind_speed is not None:
            print(f"Temperature: {temperature}°F, Wind: {wind_speed} mph")
        else:
            print("Observation data incomplete.")
    else:
        print("No observations found.")
else:
    print("No features in API response.")


   
   

Current Weather Conditions for Hennepin County:
No observations found.


## Note: 3/13 
Clearly, the observation part still isn't working correctly. I'll keep working on that.

I still need to figure out how to get this data over to the database. 